# Analyzing New York City Taxi Data with Spark Structured Streaming

## Setup

In [1]:
import os
from delta import configure_spark_with_delta_pip

import time
from pyspark.sql.window import Window

from pyspark.sql.functions import explode, lead, col, unix_timestamp, sum
from pyspark.sql.functions import split
import pyspark.sql.functions as F
from pyspark.context import SparkContext
from pyspark.conf import SparkConf
from pyspark.sql.session import SparkSession
from pyspark.sql.functions import col
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, BooleanType, TimestampType, DateType

In [2]:
builder = SparkSession.builder.appName("NYTaxiTrips") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

spark = configure_spark_with_delta_pip(builder).getOrCreate()

In [3]:
BOOTSTRAP_SERVERS = os.environ.get('BOOTSTRAP_SERVERS')
assert BOOTSTRAP_SERVERS is not None, 'BOOTSTRAP_SERVERS must be set'

TRIP_TOPIC = 'trips'
FARE_TOPIC = 'fares'

Be sure to start the stream on Kafka!

In [4]:
trip_schema = StructType(
    [
        StructField("medallion", StringType(), False),
        StructField("hack_license", StringType(), False),
        StructField("vendor_id", StringType(), False),
        StructField("rate_code", IntegerType(), False),
        StructField("store_and_fwd_flag", StringType(), False),
        StructField("pickup_datetime", TimestampType(), False),
        StructField("dropoff_datetime", TimestampType(), False),
        StructField("passenger_count", IntegerType(), False),
        StructField("trip_time_in_secs", IntegerType(), False),
        StructField("trip_distance", IntegerType(), False),
        StructField("pickup_longitude", IntegerType(), False),
        StructField("pickup_latitude", IntegerType(), False),
        StructField("dropoff_longitude", IntegerType(), False),
        StructField("dropoff_latitude", IntegerType(), False),
        StructField("timestamp", TimestampType(), False),
    ]
)

fare_schema = StructType(
    [
        StructField("medallion", StringType(), False),
        StructField("hack_license", StringType(), False),
        StructField("vendor_id", StringType(), False),
        StructField("pickup_datetime", TimestampType(), False),
        StructField("payment_type", StringType(), False),
        StructField("fare_amount", DoubleType(), False),
        StructField("surcharge", DoubleType(), False),
        StructField("mta_tax", DoubleType(), False),
        StructField("tip_amount", DoubleType(), False),
        StructField("tolls_amount", DoubleType(), False),
        StructField("total_amount", DoubleType(), False),
        StructField("timestamp", TimestampType(), False),
    ]
)

In [5]:
from pyspark.sql.functions import from_json

lines = (spark.readStream                        # Get the DataStreamReader
  .format("kafka")                                 # Specify the source format as "kafka"
  .option("kafka.bootstrap.servers", BOOTSTRAP_SERVERS) # Configure the Kafka server name and port
  .option("subscribe", TRIP_TOPIC)                       # Subscribe to the "en" Kafka topic 
  .option("startingOffsets", "earliest")           # The start point when a query is started
  .option("maxOffsetsPerTrigger", 100)             # Rate limit on max offsets per trigger interval
  .load()
)

In [6]:
query = (lines.writeStream
  .outputMode("append")
  .format("memory")
  .queryName("myQuery")
  .start())

import time
time.sleep(10)

spark.sql("SELECT * FROM myQuery").show()

+----+--------------------+-----+---------+------+--------------------+-------------+
| key|               value|topic|partition|offset|           timestamp|timestampType|
+----+--------------------+-----+---------+------+--------------------+-------------+
|NULL|[7B 22 6D 65 64 6...|trips|        0|     0|2024-06-04 17:20:...|            0|
|NULL|[7B 22 6D 65 64 6...|trips|        0|     1|2024-06-04 17:20:...|            0|
|NULL|[7B 22 6D 65 64 6...|trips|        0|     2|2024-06-04 17:20:...|            0|
|NULL|[7B 22 6D 65 64 6...|trips|        0|     3|2024-06-04 17:20:...|            0|
|NULL|[7B 22 6D 65 64 6...|trips|        0|     4|2024-06-04 17:20:...|            0|
|NULL|[7B 22 6D 65 64 6...|trips|        0|     5|2024-06-04 17:20:...|            0|
|NULL|[7B 22 6D 65 64 6...|trips|        0|     6|2024-06-04 17:20:...|            0|
|NULL|[7B 22 6D 65 64 6...|trips|        0|     7|2024-06-04 17:20:...|            0|
|NULL|[7B 22 6D 65 64 6...|trips|        0|     8|2024

## Tasks

### QUERY-1

Utilization over a window of 5, 10, and 15 minutes per taxi/driver. This can be computed by computing the idle time per taxi. How does it change? Is there an optimal window?

In [7]:
trips_cleaned_df = lines.select(from_json(col("value").cast("string"), trip_schema).alias("parsed_value"))
trips_cleaned_df = trips_cleaned_df.select("parsed_value.*")
trips_cleaned_df.printSchema()    

root
 |-- medallion: string (nullable = true)
 |-- hack_license: string (nullable = true)
 |-- vendor_id: string (nullable = true)
 |-- rate_code: integer (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_time_in_secs: integer (nullable = true)
 |-- trip_distance: integer (nullable = true)
 |-- pickup_longitude: integer (nullable = true)
 |-- pickup_latitude: integer (nullable = true)
 |-- dropoff_longitude: integer (nullable = true)
 |-- dropoff_latitude: integer (nullable = true)
 |-- timestamp: timestamp (nullable = true)



In [8]:
def create_table_if_exists(output_path,table_name):
    data_exists = False
    for _i in range(60): # you can replace this with while, currently timeouts after about 60 seconds
        try:
            time.sleep(1)
            files = os.listdir(output_path)
            for _f in files:
                if ".parquet" in _f:
                    if len(os.listdir(f"{output_path}/_delta_log"))>0:
                        print("data exists")
                        data_exists = True
                        break
            if data_exists:
                spark.sql(f"CREATE TABLE IF NOT EXISTS {table_name} USING DELTA LOCATION '{table_name}'") # table metastore is created once there is some data (.parquet) in the directory
                break
        except Exception as e:
            print(e) # if you want to see the exceptions, uncomment this
            pass

In [11]:
checkpoint_dir = 'trip_checkpoints'
table_name = 'trips'
output_dir = f"output/{table_name}"

query = (trips_cleaned_df
         .select("medallion", "pickup_datetime", "passenger_count")
         .groupBy("medallion", F.window("pickup_datetime", "5 minutes"))
         .sum("passenger_count")
         .withColumnRenamed("sum(passenger_count)", "passenger_count")
            .writeStream
            .outputMode("complete")
            .format("delta")
            .queryName(table_name)
            .option("checkpointLocation", checkpoint_dir)
            .start(output_dir)
)

create_table_if_exists(output_dir,table_name)

[Errno 2] No such file or directory: 'output/trips'
[Errno 2] No such file or directory: 'output/trips'
[Errno 2] No such file or directory: 'output/trips'
[Errno 2] No such file or directory: 'output/trips'
[Errno 2] No such file or directory: 'output/trips'
[Errno 2] No such file or directory: 'output/trips/_delta_log'
[Errno 2] No such file or directory: 'output/trips/_delta_log'
[Errno 2] No such file or directory: 'output/trips/_delta_log'
[Errno 2] No such file or directory: 'output/trips/_delta_log'
[Errno 2] No such file or directory: 'output/trips/_delta_log'
[Errno 2] No such file or directory: 'output/trips/_delta_log'
[Errno 2] No such file or directory: 'output/trips/_delta_log'
[Errno 2] No such file or directory: 'output/trips/_delta_log'
[Errno 2] No such file or directory: 'output/trips/_delta_log'
[Errno 2] No such file or directory: 'output/trips/_delta_log'
[Errno 2] No such file or directory: 'output/trips/_delta_log'
data exists


In [9]:
query = (trips_cleaned_df.writeStream
  .outputMode("append")
  .format("memory")
  .queryName("ParsedTripData")
  .start())

import time
time.sleep(10)

spark.sql("SELECT * FROM ParsedTripData").show()

+--------------------+--------------------+---------+---------+------------------+-------------------+-------------------+---------------+-----------------+-------------+----------------+---------------+-----------------+----------------+-------------------+
|           medallion|        hack_license|vendor_id|rate_code|store_and_fwd_flag|    pickup_datetime|   dropoff_datetime|passenger_count|trip_time_in_secs|trip_distance|pickup_longitude|pickup_latitude|dropoff_longitude|dropoff_latitude|          timestamp|
+--------------------+--------------------+---------+---------+------------------+-------------------+-------------------+---------------+-----------------+-------------+----------------+---------------+-----------------+----------------+-------------------+
|89D227B655E5C82AE...|BA96DE419E711691B...|      CMT|     NULL|                 N|2013-01-01 15:11:48|2013-01-01 15:18:10|           NULL|             NULL|         NULL|            NULL|           NULL|             NULL|  

In [14]:
from pyspark.sql.functions import window, col, unix_timestamp, max, min

window_durations = [5 * 60, 10 * 60, 15 * 60]

for window_duration in window_durations:
    window_col = window("pickup_datetime", "{} seconds".format(window_duration))
    df_windowed = df.groupBy("medallion", window_col).agg((max("dropoff_datetime").cast("long") - min("pickup_datetime").cast("long")).alias("busy_time"))
    
    df_windowed = df_windowed.withColumn("idle_time", window_duration - col("busy_time"))
    
    df_windowed = df_windowed.withColumn("utilization", col("busy_time") / window_duration)
    
    query = (df_windowed.writeStream
      .outputMode("complete")
      .format("memory")
      .queryName("utilization_{}_minutes".format(window_duration // 60))
      .trigger(processingTime="5 second")
      .start())

spark.sql("SELECT * FROM utilization_5_minutes").show()
spark.sql("SELECT * FROM utilization_10_minutes").show()
spark.sql("SELECT * FROM utilization_15_minutes").show()

+---------+------+---------+---------+-----------+
|medallion|window|busy_time|idle_time|utilization|
+---------+------+---------+---------+-----------+
+---------+------+---------+---------+-----------+

+---------+------+---------+---------+-----------+
|medallion|window|busy_time|idle_time|utilization|
+---------+------+---------+---------+-----------+
+---------+------+---------+---------+-----------+

+---------+------+---------+---------+-----------+
|medallion|window|busy_time|idle_time|utilization|
+---------+------+---------+---------+-----------+
+---------+------+---------+---------+-----------+



### QUERY-2 

The average time it takes for a taxi to find its next fare(trip) per destination borough. This can be computed by finding the time difference, e.g. in seconds, between the trip's drop off and the next trip's pick up within a given unit of time

In [ ]:
# remember you can register another stream


### QUERY-3

The number of trips that started and ended within the same borough in the last hour

In [ ]:
# remember you can register another stream


### QUERY-4 

The number of trips that started in one borough and ended in another one in the last hour